In [48]:
import os

import pandas as pd
import numpy as np
import collections

import json

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from dateutil import parser

from sklearn import preprocessing
from tqdm import tqdm

from joblib import Parallel, delayed

import pickle

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Task 1: Parse data

In [3]:
path_to_datas = './datas'

In [4]:
class Payload(object):
    def __init__(self, j):
        self.__dict__ = json.loads(j)

#### import data

In [5]:
data_list = []

for root, dirs, files in os.walk(path_to_datas):
    for filename in files:
        print(filename)
        path = f'./{path_to_datas}/{filename}'
        with open(path, "r", encoding='utf-8') as f:
            data = f.read()
            payload = Payload(data)
            data_list.append(pd.DataFrame(payload.features))

altaiskii-krai.geojson
amurskaia-oblast.geojson
arkhangelskaia-oblast.geojson
astrakhanskaia-oblast.geojson
belgorodskaia-oblast.geojson
brianskaia-oblast.geojson
evreiskaia-avtonomnaia-oblast.geojson
irkutskaia-oblast.geojson
ivanovskaia-oblast.geojson
kabardino-balkarskaia-respublika.geojson
kaliningradskaia-oblast.geojson
kaluzhskaia-oblast.geojson
kamchatskii-krai.geojson
karachaevo-cherkesskaia-respublika.geojson
kemerovskaia-oblast-kuzbass.geojson
kirovskaia-oblast.geojson
kostromskaia-oblast.geojson
krasnodarskii-krai.geojson
kurganskaia-oblast.geojson
kurskaia-oblast.geojson
leningradskaia-oblast.geojson
lipetskaia-oblast.geojson
magadanskaia-oblast.geojson
moskovskaia-oblast.geojson
moskva.geojson
vladimirskaia-oblast.geojson
volgogradskaia-oblast.geojson
vologodskaia-oblast.geojson
voronezhskaia-oblast.geojson
zabaikalskii-krai.geojson


#### concatinate all data in one dataset

In [6]:
history_df = pd.concat(data_list)
history_df.head()

,type,geometry,properties
0,Feature,"{'type': 'Point', 'coordinates': [83.6903, 53....","{'id': 2311491, 'tags': ['Дорожно-транспортные..."
1,Feature,"{'type': 'Point', 'coordinates': [83.699802, 5...","{'id': 2489516, 'tags': ['Дорожно-транспортные..."
2,Feature,"{'type': 'Point', 'coordinates': [83.704684, 5...","{'id': 2565463, 'tags': ['Дорожно-транспортные..."
3,Feature,"{'type': 'Point', 'coordinates': [83.690007, 5...","{'id': 2565477, 'tags': ['Дорожно-транспортные..."
4,Feature,"{'type': 'Point', 'coordinates': [83.718964, 5...","{'id': 2565484, 'tags': ['Дорожно-транспортные..."


# Task 2: Create structure dataset

#### split geometry point feature -> coorditates X and Y (point_x & point_y)

In [7]:
history_df['point_x'] = [history_df['geometry'].iloc[i]['coordinates'][0] for i in range(history_df.shape[0])]
history_df['point_y'] = [history_df['geometry'].iloc[i]['coordinates'][1] for i in range(history_df.shape[0])]

In [8]:
history_df.head()

,type,geometry,properties,point_x,point_y
0,Feature,"{'type': 'Point', 'coordinates': [83.6903, 53....","{'id': 2311491, 'tags': ['Дорожно-транспортные...",83.690300,53.342000
1,Feature,"{'type': 'Point', 'coordinates': [83.699802, 5...","{'id': 2489516, 'tags': ['Дорожно-транспортные...",83.699802,53.369248
2,Feature,"{'type': 'Point', 'coordinates': [83.704684, 5...","{'id': 2565463, 'tags': ['Дорожно-транспортные...",83.704684,53.343391
3,Feature,"{'type': 'Point', 'coordinates': [83.690007, 5...","{'id': 2565477, 'tags': ['Дорожно-транспортные...",83.690007,53.380807
4,Feature,"{'type': 'Point', 'coordinates': [83.718964, 5...","{'id': 2565484, 'tags': ['Дорожно-транспортные...",83.718964,53.351736


#### drops features

In [9]:
history_df = history_df.drop(['type', 'geometry'], axis=1)

In [10]:
history_df.head()

,properties,point_x,point_y
0,"{'id': 2311491, 'tags': ['Дорожно-транспортные...",83.690300,53.342000
1,"{'id': 2489516, 'tags': ['Дорожно-транспортные...",83.699802,53.369248
2,"{'id': 2565463, 'tags': ['Дорожно-транспортные...",83.704684,53.343391
3,"{'id': 2565477, 'tags': ['Дорожно-транспортные...",83.690007,53.380807
4,"{'id': 2565484, 'tags': ['Дорожно-транспортные...",83.718964,53.351736


In [11]:
history_df.properties.iloc[0]

{'id': 2311491,
 'tags': ['Дорожно-транспортные происшествия'],
 'light': 'В темное время суток, освещение отсутствует',
 'point': {'lat': 53.342, 'long': 83.6903},
 'nearby': ['Многоквартирные жилые дома', 'Административные здания', 'АЗС'],
 'region': 'Барнаул',
 'scheme': '610',
 'address': 'г Барнаул, тракт Павловский, 249 а',
 'weather': ['Ясно'],
 'category': 'Наезд на препятствие',
 'datetime': '2017-09-04 22:15:00',
 'severity': 'Легкий',
 'vehicles': [{'year': 2002,
   'brand': 'TOYOTA',
   'color': 'Черный',
   'model': 'Camry',
   'category': 'С-класс (малый средний, компактный) до 4,3 м',
   'participants': [{'role': 'Водитель',
     'gender': 'Мужской',
     'violations': ['Другие нарушения ПДД водителем',
      'Нарушение водителем правил применения ремней безопасности (ставится в случае, когда не пристегнут пассажир)'],
     'health_status': 'Не пострадал',
     'years_of_driving_experience': None},
    {'role': 'Пассажир',
     'gender': 'Женский',
     'violations': ['Н

#### drops features

In [12]:
exceptions = ['vehicles', 'id', 'point', 'participants', 'scheme']

#### first idea organize sructure

In [13]:
new_datas = []

for row in history_df.properties.values:
    new_row = {}
    for key in row:
        if key not in exceptions:
            new_row[key] = row[key]
    new_datas.append(new_row)
        
new_datas[0]

{'tags': ['Дорожно-транспортные происшествия'],
 'light': 'В темное время суток, освещение отсутствует',
 'nearby': ['Многоквартирные жилые дома', 'Административные здания', 'АЗС'],
 'region': 'Барнаул',
 'address': 'г Барнаул, тракт Павловский, 249 а',
 'weather': ['Ясно'],
 'category': 'Наезд на препятствие',
 'datetime': '2017-09-04 22:15:00',
 'severity': 'Легкий',
 'dead_count': 0,
 'injured_count': 1,
 'parent_region': 'Алтайский край',
 'road_conditions': ['Сухое'],
 'participants_count': 2,
 'participant_categories': ['Все участники']}

In [14]:
df = pd.DataFrame(new_datas)
df['point_x'] = history_df['point_x'].values
df['point_y'] = history_df['point_y'].values
df.head()

,tags,light,nearby,region,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,point_x,point_y
0,[Дорожно-транспортные происшествия],"В темное время суток, освещение отсутствует","[Многоквартирные жилые дома, Административные ...",Барнаул,"г Барнаул, тракт Павловский, 249 а",[Ясно],Наезд на препятствие,2017-09-04 22:15:00,Легкий,0,1,Алтайский край,[Сухое],2,[Все участники],83.690300,53.342000
1,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Нерегулируемый пе...",Барнаул,"г Барнаул, ул Юрина, 241",[Дождь],Столкновение,2020-09-24 09:30:00,Легкий,0,2,Алтайский край,[Мокрое],3,"[Дети, Все участники]",83.699802,53.369248
2,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Школа либо иная д...",Барнаул,"г Барнаул, тракт Павловский, 68",[Ясно],Наезд на пешехода,2021-04-20 18:55:00,Легкий,0,1,Алтайский край,"[Сухое, Отсутствие, плохая различимость горизо...",2,"[Дети, Пешеходы, Все участники]",83.704684,53.343391
3,[Дорожно-транспортные происшествия],Светлое время суток,"[Жилые дома индивидуальной застройки, Нерегули...",Барнаул,"г Барнаул, ул Совхозная, 10 а",[Пасмурно],Наезд на пешехода,2021-04-28 12:20:00,Легкий,0,1,Алтайский край,[Сухое],2,"[Пешеходы, Все участники]",83.690007,53.380807
4,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","[Многоквартирные жилые дома, Жилые дома индиви...",Барнаул,"г Барнаул, ул Советской Армии, 123",[Дождь],Наезд на пешехода,2021-04-27 21:00:00,Тяжёлый,0,1,Алтайский край,"[Мокрое, Отсутствие, плохая различимость гориз...",2,"[Пешеходы, Все участники]",83.718964,53.351736


#### drop not-a-number values from dataset

In [15]:
df = df.dropna()

In [16]:
df.describe()

,dead_count,injured_count,participants_count,point_x,point_y
count,417518.000000,417518.000000,417518.000000,417518.000000,417518.000000
mean,0.117207,1.254399,2.460739,51.381948,53.340157
std,0.389871,0.894528,1.163979,27.068900,4.819420
min,0.000000,0.000000,1.000000,0.000300,0.000300
25%,0.000000,1.000000,2.000000,37.547300,51.344470
50%,0.000000,1.000000,2.000000,39.271100,54.683087
75%,0.000000,1.000000,3.000000,48.019167,55.858100
max,19.000000,47.000000,54.000000,176.432500,90.000000


In [17]:
df.dtypes

tags                       object
light                      object
nearby                     object
region                     object
address                    object
weather                    object
category                   object
datetime                   object
severity                   object
dead_count                  int64
injured_count               int64
parent_region              object
road_conditions            object
participants_count          int64
participant_categories     object
point_x                   float64
point_y                   float64
dtype: object

#### convert string -> datetime

In [18]:
def to_date(x):
    return parser.parse(x)

In [19]:
df['datetime'] = df['datetime'].apply(to_date)

In [20]:
df.head()

,tags,light,nearby,region,address,weather,category,datetime,severity,dead_count,injured_count,parent_region,road_conditions,participants_count,participant_categories,point_x,point_y
0,[Дорожно-транспортные происшествия],"В темное время суток, освещение отсутствует","[Многоквартирные жилые дома, Административные ...",Барнаул,"г Барнаул, тракт Павловский, 249 а",[Ясно],Наезд на препятствие,2017-09-04 22:15:00,Легкий,0,1,Алтайский край,[Сухое],2,[Все участники],83.690300,53.342000
1,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Нерегулируемый пе...",Барнаул,"г Барнаул, ул Юрина, 241",[Дождь],Столкновение,2020-09-24 09:30:00,Легкий,0,2,Алтайский край,[Мокрое],3,"[Дети, Все участники]",83.699802,53.369248
2,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Школа либо иная д...",Барнаул,"г Барнаул, тракт Павловский, 68",[Ясно],Наезд на пешехода,2021-04-20 18:55:00,Легкий,0,1,Алтайский край,"[Сухое, Отсутствие, плохая различимость горизо...",2,"[Дети, Пешеходы, Все участники]",83.704684,53.343391
3,[Дорожно-транспортные происшествия],Светлое время суток,"[Жилые дома индивидуальной застройки, Нерегули...",Барнаул,"г Барнаул, ул Совхозная, 10 а",[Пасмурно],Наезд на пешехода,2021-04-28 12:20:00,Легкий,0,1,Алтайский край,[Сухое],2,"[Пешеходы, Все участники]",83.690007,53.380807
4,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","[Многоквартирные жилые дома, Жилые дома индиви...",Барнаул,"г Барнаул, ул Советской Армии, 123",[Дождь],Наезд на пешехода,2021-04-27 21:00:00,Тяжёлый,0,1,Алтайский край,"[Мокрое, Отсутствие, плохая различимость гориз...",2,"[Пешеходы, Все участники]",83.718964,53.351736


In [21]:
df.dtypes

tags                              object
light                             object
nearby                            object
region                            object
address                           object
weather                           object
category                          object
datetime                  datetime64[ns]
severity                          object
dead_count                         int64
injured_count                      int64
parent_region                     object
road_conditions                   object
participants_count                 int64
participant_categories            object
point_x                          float64
point_y                          float64
dtype: object

# Task 3: Labeling

#### create list all feature where dtype is object and value is type list

In [22]:
cols_with_type_list = ['tags', 'nearby', 'weather', 'road_conditions', 'participant_categories']

In [23]:
uniques = {}

for col in cols_with_type_list:
    rrd = []
    for value in df[col].values:
        for item in value:
            rrd.append(item)
    uniques[col] = list(set(rrd))

#### string -> number

In [24]:
labeling = {}

for col in cols_with_type_list:
    labeling[col] = preprocessing.LabelEncoder()
    labeling[col].fit(uniques[col])

In [25]:
labeling

{'tags': LabelEncoder(),
 'nearby': LabelEncoder(),
 'weather': LabelEncoder(),
 'road_conditions': LabelEncoder(),
 'participant_categories': LabelEncoder()}

In [26]:
sample_labels = {}

for col in cols_with_type_list:
    sample_labels[col] = {}
    for uniq in uniques[col]:
        sample_labels[col][uniq] = labeling[col].transform([uniq])[0]

In [27]:
sample_labels

{'tags': {'Дорожно-транспортные происшествия': 2,
  'ДТП и пострадавшие пешеходы, из-за наезда на пешеходов, на пешеходных переходах, по вине пешеходов': 1,
  'ДТП и пострадавшие дети в возрасте до 16 лет': 0},
 'nearby': {'Иное образовательное учреждение': 11,
  'Нерегулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иной детской организации': 33,
  'Подход к мосту, эстакаде, путепроводу': 43,
  'Объект торговли, общественного питания на автодороге вне НП': 36,
  'Регулируемый пешеходный переход, расположенный на участке улицы или дороги, проходящей вдоль территории школы или иного детского учреждения': 51,
  'Иной объект': 12,
  'Внутридворовая территория': 6,
  'Мост, эстакада, путепровод': 22,
  'Нерегулируемое пересечение с круговым движением': 24,
  'Подземный пешеходный переход': 42,
  'Школа либо иное детское (в т.ч. дошкольное) учреждение': 60,
  'Производственное предприятие': 44,
  'Крупный торговый объект (являющи

In [28]:
for col in cols_with_type_list:
    for label in sample_labels[col].values():
        df[f'{col}-{label}'] = [0 for i in range(df.shape[0])]

In [29]:
df.head()

,tags,light,nearby,region,address,weather,category,datetime,severity,dead_count,...,road_conditions-37,road_conditions-19,road_conditions-40,road_conditions-27,participant_categories-0,participant_categories-5,participant_categories-2,participant_categories-3,participant_categories-4,participant_categories-1
0,[Дорожно-транспортные происшествия],"В темное время суток, освещение отсутствует","[Многоквартирные жилые дома, Административные ...",Барнаул,"г Барнаул, тракт Павловский, 249 а",[Ясно],Наезд на препятствие,2017-09-04 22:15:00,Легкий,0,...,0,0,0,0,0,0,0,0,0,0
1,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Нерегулируемый пе...",Барнаул,"г Барнаул, ул Юрина, 241",[Дождь],Столкновение,2020-09-24 09:30:00,Легкий,0,...,0,0,0,0,0,0,0,0,0,0
2,"[Дорожно-транспортные происшествия, ДТП и пост...",Светлое время суток,"[Многоквартирные жилые дома, Школа либо иная д...",Барнаул,"г Барнаул, тракт Павловский, 68",[Ясно],Наезд на пешехода,2021-04-20 18:55:00,Легкий,0,...,0,0,0,0,0,0,0,0,0,0
3,[Дорожно-транспортные происшествия],Светлое время суток,"[Жилые дома индивидуальной застройки, Нерегули...",Барнаул,"г Барнаул, ул Совхозная, 10 а",[Пасмурно],Наезд на пешехода,2021-04-28 12:20:00,Легкий,0,...,0,0,0,0,0,0,0,0,0,0
4,[Дорожно-транспортные происшествия],"В темное время суток, освещение включено","[Многоквартирные жилые дома, Жилые дома индиви...",Барнаул,"г Барнаул, ул Советской Армии, 123",[Дождь],Наезд на пешехода,2021-04-27 21:00:00,Тяжёлый,0,...,0,0,0,0,0,0,0,0,0,0


#### fill all dummies feature

operation time 35min

In [32]:
def mark_data(idx, value, col):
    for item in value:
        df[f'{col}-{sample_labels[col][item]}'].iloc[idx] = 1
    if (idx % 50000 == 0):
        print(f'{col}: {idx} / {df.shape[0]}')

for col in cols_with_type_list:
    for idx, value in enumerate(df[col].values):
        mark_data(idx, value, col)

tags: 0 / 417518
tags: 50000 / 417518
tags: 100000 / 417518
tags: 150000 / 417518
tags: 200000 / 417518
tags: 250000 / 417518
tags: 300000 / 417518
tags: 350000 / 417518
tags: 400000 / 417518
nearby: 0 / 417518
nearby: 50000 / 417518
nearby: 100000 / 417518
nearby: 150000 / 417518
nearby: 200000 / 417518
nearby: 250000 / 417518
nearby: 300000 / 417518
nearby: 350000 / 417518
nearby: 400000 / 417518
weather: 0 / 417518
weather: 50000 / 417518
weather: 100000 / 417518
weather: 150000 / 417518
weather: 200000 / 417518
weather: 250000 / 417518
weather: 300000 / 417518
weather: 350000 / 417518
weather: 400000 / 417518
road_conditions: 0 / 417518
road_conditions: 50000 / 417518
road_conditions: 100000 / 417518
road_conditions: 150000 / 417518
road_conditions: 200000 / 417518
road_conditions: 250000 / 417518
road_conditions: 300000 / 417518
road_conditions: 350000 / 417518
road_conditions: 400000 / 417518
participant_categories: 0 / 417518
participant_categories: 50000 / 417518
participant_ca

In [33]:
df = df.drop(cols_with_type_list, axis=1)

In [34]:
df.head()

,light,region,address,category,datetime,severity,dead_count,injured_count,parent_region,participants_count,...,road_conditions-37,road_conditions-19,road_conditions-40,road_conditions-27,participant_categories-0,participant_categories-5,participant_categories-2,participant_categories-3,participant_categories-4,participant_categories-1
0,"В темное время суток, освещение отсутствует",Барнаул,"г Барнаул, тракт Павловский, 249 а",Наезд на препятствие,2017-09-04 22:15:00,Легкий,0,1,Алтайский край,2,...,0,0,1,0,0,0,0,0,0,1
1,Светлое время суток,Барнаул,"г Барнаул, ул Юрина, 241",Столкновение,2020-09-24 09:30:00,Легкий,0,2,Алтайский край,3,...,0,0,0,0,0,0,1,0,0,1
2,Светлое время суток,Барнаул,"г Барнаул, тракт Павловский, 68",Наезд на пешехода,2021-04-20 18:55:00,Легкий,0,1,Алтайский край,2,...,0,0,1,0,0,1,1,0,0,1
3,Светлое время суток,Барнаул,"г Барнаул, ул Совхозная, 10 а",Наезд на пешехода,2021-04-28 12:20:00,Легкий,0,1,Алтайский край,2,...,0,0,1,0,0,1,0,0,0,1
4,"В темное время суток, освещение включено",Барнаул,"г Барнаул, ул Советской Армии, 123",Наезд на пешехода,2021-04-27 21:00:00,Тяжёлый,0,1,Алтайский край,2,...,0,0,0,0,0,1,0,0,0,1


## Start labeling other features

In [38]:
for_labeling = ['light', 'region', 'address', 'severity', 'parent_region', 'category']

### mini task: preprocessing address

In [39]:
def preprocessing_address(item):
    result = item
    address = item.split(',')
    if len(address) > 1 and address[1] != ' ':
        result = address[1]
    return result

df['address'] = df['address'].apply(lambda item: preprocessing_address(item))

In [43]:
for col in for_labeling:
    labeling[col] = preprocessing.LabelEncoder()
    labeling[col].fit(df[col].values)
    df[col] = labeling[col].transform(df[col].values)

In [44]:
df.head()

,light,region,address,category,datetime,severity,dead_count,injured_count,parent_region,participants_count,...,road_conditions-37,road_conditions-19,road_conditions-40,road_conditions-27,participant_categories-0,participant_categories-5,participant_categories-2,participant_categories-3,participant_categories-4,participant_categories-1
0,2,42,8706,10,2017-09-04 22:15:00,0,0,1,0,2,...,0,0,1,0,0,0,0,0,0,1
1,4,42,16864,16,2020-09-24 09:30:00,0,0,2,0,3,...,0,0,0,0,0,0,1,0,0,1
2,4,42,8706,9,2021-04-20 18:55:00,0,0,1,0,2,...,0,0,1,0,0,1,1,0,0,1
3,4,42,15435,9,2021-04-28 12:20:00,0,0,1,0,2,...,0,0,1,0,0,1,0,0,0,1
4,0,42,15427,9,2021-04-27 21:00:00,2,0,1,0,2,...,0,0,0,0,0,1,0,0,0,1


In [45]:
df.dtypes

light                                int32
region                               int32
address                              int32
category                             int32
datetime                    datetime64[ns]
                                 ...      
participant_categories-5             int64
participant_categories-2             int64
participant_categories-3             int64
participant_categories-4             int64
participant_categories-1             int64
Length: 133, dtype: object

## All data of number type and count nan = 0

# Task 4: Export created dataset

In [46]:
df.to_csv("dataframe.csv")

export labeling file

In [49]:
outfile = open('labeling.pkl','wb')
pickle.dump(labeling, outfile)
outfile.close()